Let's Create Our First Agent Using `smolagents` 

In the last section, we learned how we can create Agents from scratch using Python code, and we saw just how tedious that process can be. Fortunately, many Agent libraries simplify this work by **handling much of the heavy lifting for you.**

In this tutorial, you'll create your very first Agent capable of performing actions such as `image generation`, `web search`, `time zone checking` and much more...

You will also publish your agent on a `Hugging Face Space` so you can share it with friends and colleagues. 

### What is `smolagents` ?

To make this Agent, we're going to use `smolagents`, a library that provides a framework for developing your agents with ease.

This lightweight library is designed for simplicity, but it abstracts away much of the complexity of building an Agent, allowing you to focus on designing your agent's behavior. 

- In short, `smolagents` is a library that focus on `CodeAgent`, a kind of agent that performs **"Actions"** through code blocks, and then **"observes"** results by executing the code. 

For Example: 

* We provided our agent with an `Image Generation Tool` and asked it to generate an image of a cat. 
* The agent inside `smolagent` is going to have the same behaviors as the custom one we built previously: it's going to think, act and observe in cycle until it reaches a final answer:



#### 1. Importing Required Libraries

In [2]:
import datetime
import pytz
import yaml
import requests

from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, load_tool, tool
from langchain_ollama.llms import OllamaLLM

c:\Users\karth\Programming\Agents\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Tools

In [3]:
@tool 
def my_custom_tool(arg1: str, arg2: int) -> str: # It's important to specify the return type.
    """A tool that does nothing yet...
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"


@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"


The **Tools** are what we are encouraging you to build in this section! We give two examples:

1. A **non-working dummy Tool** that you can modify to make something useful. 
2. An **Actually working Tool**, that gets the current time somewhere in the world.

To define your tool it is important to: 

1. Provide input and output types for your function, like `get_current_time_in_timezone(timezone: str) -> str:`
2. A **Well formatted docstring**: `smolagents` is expecting all the arguments to have a **textual description in the docstring.**

How this tool works:

In [4]:
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone. 
    Args: 
        timezone: A string representing a valid timezone (e.g:, "America/New york").
    """
    try: 
        # Create timezone object
        tz = pytz.timezone(timezone)
        
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone {timezone}: {str(e)}"

In [14]:
tz = pytz.timezone("Asia/Kolkata")

tz

<DstTzInfo 'Asia/Kolkata' LMT+5:53:00 STD>

In [15]:
get_current_time_in_timezone("Asia/Kolkata")

'The current local time in Asia/Kolkata is: 2025-06-18 19:44:30'

#### 4. LLM Model (Coder Model)

In [5]:
model = OllamaLLM(model="qwen2.5-coder:3b")


res = model.invoke("write a python code on fibonaci series")

print(res)

Certainly! Below is a simple Python function to generate the Fibonacci series up to a specified number of terms:

```python
def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]

    fib_series = [0, 1]
    for i in range(2, n):
        next_value = fib_series[i - 1] + fib_series[i - 2]
        fib_series.append(next_value)

    return fib_series

# Example usage:
num_terms = 10
print(f"Fibonacci series up to {num_terms} terms: {fibonacci(num_terms)}")
```

This function takes an integer `n` as input and returns a list containing the Fibonacci series up to the `n`th term. You can change the value of `num_terms` to generate more or fewer terms in the series.


#### 5. The Agent

It uses `Qwen-2.5-Coder-3b` as the LLM engine.

In [4]:
final_answer = FinalAnswerTool()

model = HfApiModel(
    max_tokens=2096,
    temperature=0.5,
    model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    custom_role_conversions=None,
)

# prompt template 
with open("prompts.yaml", 'r') as stream: 
    prompt_templates = yaml.safe_load(stream)

In [5]:
prompt_templates

{'system_prompt': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_ans

In [8]:
agent = CodeAgent(
    tools=[get_current_time_in_timezone, final_answer],
    model=model, 
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

Using agent for math calculation: 

In [7]:
agent.run(task="what is the result of 1 + 1.00009 - 2.666 ?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is the result of 1 + 1.00009 - 2.666 ?                                                                     │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 1 + 1.00009 - 2.666                                                                                     
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: -0.6659099999999998

[Step 1: Duration 4.50 seconds| Input tokens: 2,033 | Output tokens: 84]

-0.6659099999999998

Using `get_current_time_of_timezone` tool 

In [9]:
agent.run("what is current time in Kolkota ?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is current time in Kolkota ?                                                                               │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  kolkata_time = get_current_time_in_timezone(timezone='Asia/Kolkata')                                             
  print(kolkata_time)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The current local time in Asia/Kolkata is: 2025-06-18 20:25:21

Out: None

[Step 1: Duration 3.88 seconds| Input tokens: 2,082 | Output tokens: 65]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  kolkata_time = "2025-06-18 20:25:21"                                                                             
  final_answer(kolkata_time)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2025-06-18 20:25:21

[Step 2: Duration 4.46 seconds| Input tokens: 4,348 | Output tokens: 158]

'2025-06-18 20:25:21'